<a href="https://colab.research.google.com/github/tonystz/cuda/blob/main/poc/gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/tonystz/cuda.git

In [2]:
!python cuda/poc/raw.py

[step0]preprocess raw log: total=1153965 skip=12507 1.6371381282806396


In [3]:
!python cuda/poc/cpu.py

[step1]reload data: 1.421327829360962 (1141458,) |S44 1141458 [b'::1-200' b'::1-200' b'::1-404' ... b'180.163.28.55-200'
 b'180.163.28.55-200' b'180.163.28.55-200']
[step2]404 data filter: 0.31173181533813477 202448
[step3]check result: 0.08201265335083008 ['216.244.66.200', '217.174.192.132', '66.249.79.234', '66.249.79.238', '66.249.79.236', '23.96.16.211', '47.101.154.149', '47.101.149.21', '38.54.24.118']


In [ ]:
!pip install pycuda # install cuda
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np